# BlazingSQL + cuML NYC Taxi Cab Fare Prediction

This demo uses pubically availible [NYC Taxi Cab Data](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction) to predict the total fare of a taxi ride in New York City given the pickup and dropoff locations. 

In this notebook, we will cover: 
- How to read and query csv files with cuDF and BlazingSQL.
- How to implement a linear regression model with cuML.

#### BlazingSQL install check
The next cell checks to determine if you have BlazingSQL installed.  If you do not have BlazingSQL installed, please first install RAPIDS and BlazingSQL via your preferred installation method (Docker or conda) from our [Release Selector](https://rapids.ai/start.html#rapids-release-selector). 

In [1]:
import sys 
#  point import path notebooks-contrib/utils
sys.path.append('../../utils/')
from sql_check import bsql_start
#  check that BlazingSQL is installed
bsql_start()

"You've got BlazingSQL set up perfectly! Let's get started with SQL in RAPIDS AI!"

## Imports

In [2]:
import cudf
from cuml import LinearRegression
from blazingsql import BlazingContext

## Create BlazingContext
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.

In [3]:
bc = BlazingContext()

BlazingContext ready


### Download Data
For this demo we will train our model with 20,000,000 rows of data from 4 CSV files (5,000,000 rows each). 

The cell below will download them from AWS to the main `notebooks-contrib/data/blazingsql/` folder for you.

In [4]:
import os
import urllib.request

# relative path to data folder
data_dir = '../../utils/blazingsql/'
# does folder exist?
if not os.path.exists(data_dir):
    print('creating blazingsql directory')
    # create folder
    os.system('mkdir ../../utils/blazingsql')

In [5]:
!ls ../../utils/

blazingsql    __pycache__      sql_check.py
env-check.py  rapids-colab.sh  update_pyarrow.py


In [6]:
# download taxi data
base_url = 'https://blazingsql-colab.s3.amazonaws.com/taxi_data/'
# thanks to Taurean Dyer
years = list(range(0, 3))
for i in range(0, 4):
    fn = 'taxi_0' + str(i) + '.csv'
    if not os.path.isfile(data_dir + fn):
        print(f'Downloading {base_url + fn} to {data_dir + fn}')
        urllib.request.urlretrieve(base_url + fn, data_dir + fn)

## Extract, transform, load
In order to train our Linear Regression model, we must first preform ETL so to prepare our data.

### ETL: Read and Join CSVs

In [7]:
# set column names and types
col_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
col_types = ['date64', 'float32', 'float32', 'float32', 
                'float32', 'float32', 'float32']

# load first csv 
gdf_00 = cudf.read_csv(data_dir+'taxi_00.csv', names=col_names, dtype=col_types)
# load second csv
gdf_01 = cudf.read_csv(data_dir+'taxi_01.csv', names=col_names, dtype=col_types)
# load third csv
gdf_02 = cudf.read_csv(data_dir+'taxi_02.csv', names=col_names, dtype=col_types)
# load fourth csv
gdf_03 = cudf.read_csv(data_dir+'taxi_03.csv', names=col_names, dtype=col_types)

# combine all those dataframes into one master dataframe
gdf = cudf.concat([gdf_00,gdf_01, gdf_02, gdf_03])

# what's it look like?
gdf.head()

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-02-02 22:30:19.002,8.9,-73.988708,40.758804,-73.986519,40.737202,1.0
1,2014-09-20 07:19:24.001,4.0,-73.990204,40.746708,-73.994728,40.750515,1.0
2,2013-02-23 07:18:05.001,5.5,-74.016762,40.709438,-74.009003,40.719498,3.0
3,2015-04-18 23:49:27.009,13.5,-74.002708,40.733730,-73.986099,40.734776,1.0
4,2010-03-04 08:15:59.001,10.5,-73.988365,40.737663,-74.012459,40.713932,1.0


In [8]:
# delete used dataframes gdf00 etc 
del gdf_00
del gdf_01
del gdf_02
del gdf_03




In [9]:
gdf.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,2.000000e+07,2.000000e+07,2.000000e+07,1.999987e+07,1.999987e+07,2.000000e+07
mean,1.134995e+01,-7.250857e+01,3.992013e+01,-7.251212e+01,3.992170e+01,1.685199e+00
std,2.878444e+01,1.273661e+01,9.756748e+00,1.273016e+01,9.628169e+00,1.334611e+00
min,-1.760000e+02,-3.442060e+03,-3.475482e+03,-3.440697e+03,-3.475482e+03,0.000000e+00
25%,6.000000e+00,-7.399206e+01,4.073494e+01,-7.399139e+01,4.073404e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075267e+01,-7.398015e+01,4.075317e+01,1.000000e+00
75%,1.250000e+01,-7.396707e+01,4.076714e+01,-7.396367e+01,4.076811e+01,2.000000e+00
max,9.396336e+04,3.456222e+03,3.408790e+03,3.456222e+03,3.537133e+03,2.080000e+02


### ETL: Create Table

In [10]:
%time
# make a table from the combined df
bc.create_table('train_taxi', gdf)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


### ETL: Query Tables for Training Data

In [11]:
# extract time columns, long & lat, # riders (all floats)
query = '''
        select 
            cast(hour(key) as float) hours, 
            cast(dayofmonth(key) as float) days, 
            cast(month(key) as float) months, 
            cast(year(key) - 2000 as float) years,  
            cast(dropoff_longitude - pickup_longitude as float) longitude_distance, 
            cast(dropoff_latitude - pickup_latitude as float) latitude_distance, 
            cast(passenger_count as float) passenger_count
        from 
            train_taxi
            '''

# run query on table (returns cuDF DataFrame)
X_train = bc.sql(query)

# fill null values 
X_train['longitude_distance'] = X_train['longitude_distance'].fillna(0)
X_train['latitude_distance'] = X_train['latitude_distance'].fillna(0)
X_train['passenger_count'] = X_train['passenger_count'].fillna(0)

# how's it look? 
X_train.head()

,hours,days,months,years,longitude_distance,latitude_distance,passenger_count
0,22.0,2.0,2.0,12.0,0.002190,-0.021603,1.0
1,7.0,20.0,9.0,14.0,-0.004524,0.003807,1.0
2,7.0,23.0,2.0,13.0,0.007759,0.010059,3.0
3,23.0,18.0,4.0,15.0,0.016609,0.001045,1.0
4,8.0,4.0,3.0,10.0,-0.024094,-0.023731,1.0


In [12]:
# query dependent variable y
y_train = bc.sql('SELECT fare_amount FROM train_taxi')

y_train.head()

,fare_amount
0,8.9
1,4.0
2,5.5
3,13.5
4,10.5


## Linear Regression
### LR: Train Model

In [13]:
%%time
# call & create cuML model
lr = LinearRegression(fit_intercept=True, normalize=False, algorithm="eig")

# train Linear Regression model 
reg = lr.fit(X_train, y_train)

# display results
print(f"Coefficients:\n{reg.coef_}\n")
print(f"Y intercept:\n{reg.intercept_}\n")

Coefficients:
0   -0.027293
1    0.003330
2    0.106819
3    0.637570
4    0.000871
5   -0.000516
6    0.092438
dtype: float32

Y intercept:
3.356637954711914

CPU times: user 421 ms, sys: 120 ms, total: 541 ms
Wall time: 540 ms


### LR: Use Model to Predict Future Taxi Fares 

#### Download Test Data
The cell below will check to see if you've already got the Test data, and, if you don't, will download it for you.

In [14]:
# do we have Test taxi file?
if not os.path.isfile('../../utils/blazingsql/test.csv'):
    !wget -P ../../data/blazingsql https://blazingsql-demos.s3-us-west-1.amazonaws.com/test.csv

--2021-04-09 07:34:59--  https://blazingsql-demos.s3-us-west-1.amazonaws.com/test.csv
Resolving blazingsql-demos.s3-us-west-1.amazonaws.com (blazingsql-demos.s3-us-west-1.amazonaws.com)... 52.219.112.217
Connecting to blazingsql-demos.s3-us-west-1.amazonaws.com (blazingsql-demos.s3-us-west-1.amazonaws.com)|52.219.112.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 982916 (960K) [text/csv]
Saving to: ‘../../data/blazingsql/test.csv.5’

test.csv.5          100%[===================>] 959.88K  2.64MB/s    in 0.4s    

2021-04-09 07:35:00 (2.64 MB/s) - ‘../../data/blazingsql/test.csv.5’ saved [982916/982916]



We are going to create this table directly from CSV, BlazingSQL requires the full path to the data for table creation. This cell uses the `pwd` bash command to identify the path to this directory, then adds the path to the notebooks-contrib `data/` directory to provide a full path to the test data.

In [15]:
# identify path to this notebook, !pwd returns SList w/ path (str) at 0th index
path = !pwd
# extract path community_tutorials_and_guides/blazingsql then
path = path[0].split('community_tutorials_and_guides/blazingsql')[0] 
# add path to data from there
path = path + '/data/blazingsql/' + 'test.csv'
# how's it look?
path

'/rapids/notebooks-contrib//data/blazingsql/test.csv'

In [16]:
# set column names and types
col_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
col_types = ['date64', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32']

# create test table directly from CSV - this doesnt make sense
bc.create_table('test_taxi', path, names=col_names, dtype=col_types)


In [17]:
# extract time columns, long & lat, # riders (all floats)
query = '''
        select 
            cast(hour(key) as float) hours, 
            cast(dayofmonth(key) as float) days, 
            cast(month(key) as float) months, 
            cast(year(key) - 2000 as float) years,  
            cast(dropoff_longitude - pickup_longitude as float) longitude_distance, 
            cast(dropoff_latitude - pickup_latitude as float) latitude_distance, 
            cast(passenger_count as float) passenger_count
        from 
            test_taxi
            '''

# run query on table (returns cuDF DataFrame)
X_test = bc.sql(query)

# fill null values 
X_test['longitude_distance'] = X_test['longitude_distance'].fillna(0)
X_test['latitude_distance'] = X_test['latitude_distance'].fillna(0)
X_test['passenger_count'] = X_test['passenger_count'].fillna(0)

# how's it look? 
X_test.head()

,hours,days,months,years,longitude_distance,latitude_distance,passenger_count
0,13.0,27.0,1.0,15.0,-0.008110,-0.019970,1.0
1,13.0,27.0,1.0,15.0,-0.012024,0.019814,1.0
2,11.0,8.0,10.0,11.0,0.002869,-0.005119,1.0
3,21.0,1.0,12.0,12.0,-0.009277,-0.016178,1.0
4,21.0,1.0,12.0,12.0,-0.022537,-0.045345,1.0


In [18]:
# predict fares 
predictions = lr.predict(X_test)

# display predictions
predictions

0       12.854544
1       12.854520
2       11.256961
3       11.811929
4       11.811933
          ...    
9909    13.714720
9910    13.157619
9911    13.419721
9912    13.657573
9913    13.259460
Length: 9914, dtype: float32

In [19]:
# add predictions to test dataframe
X_test['predicted_fare'] = predictions

# how's that look?
X_test.head()

,hours,days,months,years,longitude_distance,latitude_distance,passenger_count,predicted_fare
0,13.0,27.0,1.0,15.0,-0.008110,-0.019970,1.0,12.854544
1,13.0,27.0,1.0,15.0,-0.012024,0.019814,1.0,12.854520
2,11.0,8.0,10.0,11.0,0.002869,-0.005119,1.0,11.256961
3,21.0,1.0,12.0,12.0,-0.009277,-0.016178,1.0,11.811929
4,21.0,1.0,12.0,12.0,-0.022537,-0.045345,1.0,11.811933
